<img src="https://raw.githubusercontent.com/Sengsathit/OCR_data_scientist_assets/main/header_pret_a_depenser.png" alt="Alternative text" />

# Introduction

# Imports

In [1]:
import warnings
import pickle
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split

import xgboost as xgb

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

import lightgbm as lgb
import gc

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# Configurer Pandas pour un affichage complet du contenu des colonnes
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Chargement des données

In [2]:
# Données issues d'un feature engineering précédent
df = pd.read_csv('../../datasets/df_train_domain.csv')

# Retenir les labels
labels = df['TARGET']

# Modélisation

## MLflow

In [3]:
# Configuration du service MLflow, rediriger vers une instance locale
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

## Pre-processing

In [4]:
# Supprimer la cible
df_transformed = df.drop(columns = ['SK_ID_CURR', 'TARGET'])

# Imputation médiane des valeurs manquantes
imputer = SimpleImputer(strategy = 'median')
imputer.fit(df_transformed)
data_transformed = imputer.transform(df_transformed)

# Normaliser chaque caractéristique entre 0 et 1
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(df_transformed)
data_transformed = scaler.transform(data_transformed)

# Sauvegarder l'imputer
with open('../tools/imputer.pkl', 'wb') as i:
    pickle.dump(imputer, i)

# Sauvegarder le scaler
with open('../tools/scaler.pkl', 'wb') as s:
    pickle.dump(scaler, s)

In [5]:
# Datasets d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data_transformed, labels, test_size=0.2, random_state=42)

## Logistic Regression

In [6]:
# Créer le modèle avec le paramètre de régularisation spécifié
logistic_regression = LogisticRegression(solver='liblinear')

# Définir la grille d'hyperparamètres
params_grid = {'C': [0.001, 0.01, 0.1, 1]}

# Configuration de GridSearchCV, avec 'roc_auc' comme métrique de scoring
grid_search = GridSearchCV(
    logistic_regression, 
    params_grid, 
    cv=5, 
    scoring='roc_auc', 
    return_train_score=True,
    n_jobs=-1
)

# Exécuter GridSearchCV
grid_search.fit(X_train, y_train)

# Pour chaque ensemble d'hyperparamètres testés, afficher les paramètres et le score associé
cv_results = grid_search.cv_results_
for i in range(len(cv_results['params'])):
    print(f"Paramètres: {cv_results['params'][i]}")
    print(f"Score AUC moyen: {cv_results['mean_test_score'][i]}")
    print(f"Score AUC d'entraînement: {cv_results['mean_train_score'][i]}")
    print('-' * 50)

# Extraire le meilleur modèle et les hyperparamètres de ce modèle
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Session MLflow
mlflow.set_experiment("Logistic Regression")

# Démarrer une nouvelle exécution pour le meilleur modèle
with mlflow.start_run(run_name="Run"):
    # Loguer les hyperparamètres du meilleur modèle
    mlflow.log_param('C', best_params['C'])

    # Calculer et loguer l'AUC pour le train et le test
    y_pred_train_proba = best_model.predict_proba(X_train)[:, 1]
    y_pred_test_proba = best_model.predict_proba(X_test)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_train_proba)
    auc_test = roc_auc_score(y_test, y_pred_test_proba)
    
    mlflow.log_metric('auc_train', auc_train)
    mlflow.log_metric('auc_test', auc_test)

    # Loguer le modèle pour la meilleure configuration
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path='logistic_regression_best_model',
        signature=infer_signature(X_train, best_model.predict(X_train)),
        registered_model_name="logistic_regression_best_model"
    )

2024/09/07 00:19:19 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression' does not exist. Creating a new experiment.


Paramètres: {'C': 0.001}
Score AUC moyen: 0.7285654971544131
Score AUC d'entraînement: 0.7300105240310759
--------------------------------------------------
Paramètres: {'C': 0.01}
Score AUC moyen: 0.7416817519246457
Score AUC d'entraînement: 0.743935588693921
--------------------------------------------------
Paramètres: {'C': 0.1}
Score AUC moyen: 0.7444570996065287
Score AUC d'entraînement: 0.7476288033933193
--------------------------------------------------
Paramètres: {'C': 1}
Score AUC moyen: 0.7463279015672881
Score AUC d'entraînement: 0.7501107290752222
--------------------------------------------------


Successfully registered model 'logistic_regression_best_model'.
2024/09/07 00:19:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_regression_best_model, version 1
Created version '1' of model 'logistic_regression_best_model'.
2024/09/07 00:19:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run Run at: http://127.0.0.1:8080/#/experiments/232330005912183947/runs/9bea8eb1fe234f4f9a389254db1b5efc.
2024/09/07 00:19:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/232330005912183947.


## Random Forest

In [7]:
# Créer le modèle de Random Forest
random_forest = RandomForestClassifier()

# Définir la grille d'hyperparamètres
params_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Configuration de GridSearchCV, avec 'roc_auc' comme métrique de scoring
grid_search = GridSearchCV(
    random_forest, 
    params_grid, 
    cv=5, 
    scoring='roc_auc', 
    return_train_score=True,
    n_jobs=-1
)
# Exécuter GridSearchCV
grid_search.fit(X_train, y_train)

# Pour chaque ensemble d'hyperparamètres testés, afficher les paramètres et le score associé
cv_results = grid_search.cv_results_
for i in range(len(cv_results['params'])):
    print(f"Paramètres: {cv_results['params'][i]}")
    print(f"Score AUC moyen: {cv_results['mean_test_score'][i]}")
    print(f"Score AUC d'entraînement: {cv_results['mean_train_score'][i]}")
    print('-' * 50)

# Extraire le meilleur modèle et les hyperparamètres de ce modèle
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Session MLflow
mlflow.set_experiment("Random Forest Classifier")

# Démarrer une nouvelle exécution pour le meilleur modèle
with mlflow.start_run(run_name="Run"):
    # Loguer les hyperparamètres du meilleur modèle
    mlflow.log_param('n_estimators', best_params['n_estimators'])
    mlflow.log_param('max_depth', best_params['max_depth'])
    mlflow.log_param('min_samples_split', best_params['min_samples_split'])

    # Calculer et loguer l'AUC pour le train et le test
    y_pred_train_proba = best_model.predict_proba(X_train)[:, 1]
    y_pred_test_proba = best_model.predict_proba(X_test)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_train_proba)
    auc_test = roc_auc_score(y_test, y_pred_test_proba)
    
    mlflow.log_metric('auc_train', auc_train)
    mlflow.log_metric('auc_test', auc_test)

    # Loguer le modèle pour la meilleure configuration
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path='random_forest_best_model',
        signature=infer_signature(X_train, best_model.predict(X_train)),
        registered_model_name="random_forest_best_model"
    )


2024/09/07 00:32:42 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest Classifier' does not exist. Creating a new experiment.


Paramètres: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50}
Score AUC moyen: 0.7169911948272073
Score AUC d'entraînement: 0.7284954091625628
--------------------------------------------------
Paramètres: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Score AUC moyen: 0.7214233703440662
Score AUC d'entraînement: 0.7324714077134049
--------------------------------------------------
Paramètres: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200}
Score AUC moyen: 0.7210800119943027
Score AUC d'entraînement: 0.732836906653123
--------------------------------------------------
Paramètres: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50}
Score AUC moyen: 0.7207339193501088
Score AUC d'entraînement: 0.7315920857106807
--------------------------------------------------
Paramètres: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
Score AUC moyen: 0.7208005571336474
Score AUC d'entraînement: 0.7323280729425491
------------------------

Successfully registered model 'random_forest_best_model'.
2024/09/07 00:32:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_best_model, version 1
Created version '1' of model 'random_forest_best_model'.
2024/09/07 00:32:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run Run at: http://127.0.0.1:8080/#/experiments/665918049452046265/runs/b48509599d874224977cf432079b0787.
2024/09/07 00:32:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/665918049452046265.


## XGBoost

In [8]:
# Créer le modèle XGBoost
xgboost_model = xgb.XGBClassifier()

# Définir la grille d'hyperparamètres
params_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Configuration de GridSearchCV avec exécution parallèle
grid_search = GridSearchCV(
    xgboost_model, 
    params_grid, 
    cv=5, 
    scoring='roc_auc', 
    return_train_score=True,
    n_jobs=-1  # Utilise tous les cœurs disponibles pour exécuter en parallèle
)

# Exécuter GridSearchCV
grid_search.fit(X_train, y_train)

# Pour chaque ensemble d'hyperparamètres testés, afficher les paramètres et le score associé
cv_results = grid_search.cv_results_
for i in range(len(cv_results['params'])):
    print(f"Paramètres: {cv_results['params'][i]}")
    print(f"Score AUC moyen: {cv_results['mean_test_score'][i]}")
    print(f"Score AUC d'entraînement: {cv_results['mean_train_score'][i]}")
    print('-' * 50)

# Extraire le meilleur modèle et les hyperparamètres de ce modèle
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Session MLflow
mlflow.set_experiment("XGBoost Classifier")

# Démarrer une nouvelle exécution pour le meilleur modèle
with mlflow.start_run(run_name="Run"):
    # Loguer les hyperparamètres du meilleur modèle
    mlflow.log_param('n_estimators', best_params['n_estimators'])
    mlflow.log_param('max_depth', best_params['max_depth'])
    mlflow.log_param('learning_rate', best_params['learning_rate'])
    mlflow.log_param('subsample', best_params['subsample'])

    # Calculer et loguer l'AUC pour le train et le test
    y_pred_train_proba = best_model.predict_proba(X_train)[:, 1]
    y_pred_test_proba = best_model.predict_proba(X_test)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_train_proba)
    auc_test = roc_auc_score(y_test, y_pred_test_proba)
    
    mlflow.log_metric('auc_train', auc_train)
    mlflow.log_metric('auc_test', auc_test)

    # Loguer le modèle pour la meilleure configuration
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path='xgboost_best_model',
        signature=infer_signature(X_train, best_model.predict(X_train)),
        registered_model_name="xgboost_best_model"
    )

2024/09/07 00:37:41 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost Classifier' does not exist. Creating a new experiment.


Paramètres: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
Score AUC moyen: 0.7053035814729915
Score AUC d'entraînement: 0.7073376206950306
--------------------------------------------------
Paramètres: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
Score AUC moyen: 0.703396162114429
Score AUC d'entraînement: 0.705940358666503
--------------------------------------------------
Paramètres: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Score AUC moyen: 0.7126580070491586
Score AUC d'entraînement: 0.7152389347520979
--------------------------------------------------
Paramètres: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Score AUC moyen: 0.7118445982082127
Score AUC d'entraînement: 0.7146975727271284
--------------------------------------------------
Paramètres: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Score AUC moyen: 0.72

Successfully registered model 'xgboost_best_model'.
2024/09/07 00:37:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgboost_best_model, version 1
Created version '1' of model 'xgboost_best_model'.
2024/09/07 00:37:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run Run at: http://127.0.0.1:8080/#/experiments/579384878940766092/runs/de55d5e14ffb41b7938dc46903ed03a7.
2024/09/07 00:37:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579384878940766092.


### Interprétation du modèle : Feature Importances

In [9]:
def plot_feature_importances(df):
    """
    Trace les importances retournées par un modèle.
    
    Args:
        df (dataframe) : importances des caractéristiques. Doit contenir les 
        caractéristiques dans une colonne appelée `features` et les importances 
        dans une colonne appelée `importance`.
        
    Returns:
        Affiche un graphique des 15 caractéristiques les plus importantes.
        
        df (dataframe) : importances des caractéristiques triées par importance 
        (de la plus élevée à la plus faible) avec une colonne pour l'importance normalisée.
    """
    
    # Trier les caractéristiques en fonction de leur importance
    df = df.sort_values('importance', ascending=False).reset_index()
    
    # Normaliser les importances des caractéristiques pour qu'elles totalisent un
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Créer un graphique à barres horizontales des importances des caractéristiques
    plt.figure(figsize=(10, 6))
    ax = plt.subplot()
    
    # Il est nécessaire d'inverser l'index pour afficher les plus importantes en haut
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align='center', edgecolor='k')
    
    # Définir les graduations et étiquettes sur l'axe y
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Étiquetage du graphique
    plt.xlabel('Importance Normalisée'); plt.title('Feature Importances')
    plt.show()
    
    return df